# Geo Location

# Table of contents

1. Imports
2. Paths
3. Load and transform images
4. Create and train model
5. Extract image meta data
6. Create dataframe
7. Load to csv-file

### 1. Imports

In [1]:
import numpy as np
import os
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm
import pandas as pd
from keras.applications.vgg16 import VGG16
import tensorflow.keras.utils as image2
from keras.applications.vgg16 import preprocess_input
from sklearn.metrics import accuracy_score
from keras.layers import Input
from PIL import Image
import exifread
import requests
from tqdm import tqdm

### 2. Paths

In [2]:
CURR_DIR_PATH = os.getcwd()

# Training paths
seg_train_path = CURR_DIR_PATH + "//data//seg_train//seg_train"

train_forest_path = seg_train_path + "//forest"
train_buildings_path = seg_train_path + "//buildings"
train_glacier_path = seg_train_path + "//glacier"
train_mountain_path = seg_train_path + "//mountain"
train_sea_path = seg_train_path + "//sea"
train_street_path = seg_train_path + "//street"

# Test paths
seg_test_path = CURR_DIR_PATH + "//data//seg_test//seg_test"

test_forest_path = seg_test_path + "//forest"
test_buildings_path = seg_test_path + "//buildings"
test_glacier_path = seg_test_path + "//glacier"
test_mountain_path = seg_test_path + "//mountain"
test_sea_path = seg_test_path + "//sea"
test_street_path = seg_test_path + "//street"

# Prediction path

seg_pred_path = CURR_DIR_PATH + "//data//seg_pred//seg_pred"

# Our path

seg_our_path = CURR_DIR_PATH + "//data//seg_our//seg_our"

### 3. Load and transform images

In [3]:
def load_data(paths):
    images = []
    labels = []
    label = -1
    for path in paths:
        label = label+1
        for file in tqdm(os.listdir(path)):
            image_path = os.path.join(path, file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image,(150, 150)) 
            image = image/ 255.0
            images.append(image)
            labels.append(label)

    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')

    output = (images,labels)
    return output

In [4]:
train_paths = [train_buildings_path, train_forest_path, train_glacier_path, train_mountain_path, train_sea_path, train_street_path]
test_paths = [test_buildings_path, test_forest_path, test_glacier_path, test_mountain_path, test_sea_path, test_street_path]
train_images, train_labels= load_data(train_paths)
print (f"Number of training images: {train_labels.shape[0]}")
test_images, test_labels = load_data(test_paths)
print (f"Number of testing images: {test_labels.shape[0]}")

100%|██████████| 2382/2382 [00:06<00:00, 356.60it/s]


Number of training images: 14034


100%|██████████| 501/501 [00:01<00:00, 475.31it/s]


Number of testing images: 3000


In [5]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

### 4. Create and train model

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

In [7]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=40, validation_split = 0.2)

Epoch 1/4
88/88 [==============================] - 111s 1s/step - loss: 1.3639 - accuracy: 0.5057 - val_loss: 0.9577 - val_accuracy: 0.6480
Epoch 2/4
88/88 [==============================] - 108s 1s/step - loss: 0.7993 - accuracy: 0.7104 - val_loss: 0.7713 - val_accuracy: 0.6990
Epoch 3/4
88/88 [==============================] - 111s 1s/step - loss: 0.6287 - accuracy: 0.7755 - val_loss: 0.6943 - val_accuracy: 0.7538
Epoch 4/4
88/88 [==============================] - 110s 1s/step - loss: 0.4812 - accuracy: 0.8348 - val_loss: 0.6881 - val_accuracy: 0.7613


In [42]:
def extract_gps_info(image_path):
    with open(image_path, 'rb') as image_file:
        tags = exifread.process_file(image_file)
        lat_deg = tags.get('GPS GPSLatitude', None)
        lat_ref = tags.get('GPS GPSLatitudeRef', None)
        long_deg = tags.get('GPS GPSLongitude', None)
        long_ref = tags.get('GPS GPSLongitudeRef', None)
        timestamp = tags.get('EXIF DateTimeOriginal', None)
        latitude = convert_to_degrees(lat_deg, lat_ref)
        longitude = convert_to_degrees(long_deg, long_ref)
        return (latitude, longitude, timestamp)

In [43]:
def convert_to_degrees(coordinate, ref):
    coordinate = list(coordinate.values)
    d = float(coordinate[0].num) / coordinate[0].den
    m = float(coordinate[1].num) / coordinate[1].den
    s = float(coordinate[2].num) / coordinate[2].den
    result = d + (m / 60.0) + (s / 3600.0)
    if str(ref) == 'S' or str(ref) == 'W':
        result *= -1
    return result

In [44]:
def get_location_name(latitude, longitude):
    url = f"https://nominatim.openstreetmap.org/reverse?format=json&lat={latitude}&lon={longitude}"
    response = requests.get(url)
    location_name = response.json().get('address').get('town') or response.json().get('address').get('city')
    country = response.json().get('address').get('country')
    return location_name, country

In [45]:
def processing(path):
    img = cv2.imread(path)
    test1_proc = image2.load_img(path , target_size=(150,150))
    test1_proc_1 = image2.img_to_array(test1_proc)
    t1 = np.expand_dims(test1_proc ,axis=0)
    t1 = t1/255
    pred_classes = model.predict(t1)
    pred = np.argmax(pred_classes ,axis=1)

    plt.subplot(121)
    pic = Image.open(path) 
    plt.xlabel("Original Image")
    plt.imshow(pic)
    
    
    plt.subplot(122)

    s1 = pd.Series(pred_classes.ravel() , index = category_label)
    s1.plot(kind = 'bar' ,figsize = (10,5))
    plt.xlabel("prediction")
    
    plt.show()

### 5. Extract image meta data

In [46]:
image_paths = []
longitudes = []
latitudes = []
timestamps = []
location_names = []
countries = []
categories = []
file_names = []
category_label=['buildings','forest', 'glacier','mountain','sea','street']


for file in os.listdir(seg_our_path):
    image_path = (os.path.join(seg_our_path, file))
    image_paths.append(image_path)
    img = cv2.imread(image_path)
    test1_proc = image2.load_img(image_path , target_size=(150,150))
    test1_proc_1 = image2.img_to_array(test1_proc)
    t1 = np.expand_dims(test1_proc ,axis=0)
    t1 = t1/255
    pred_classes = model.predict(t1)
    pred = np.argmax(pred_classes ,axis=1)
    for i in range(len(category_label)):
        if pred == i:
            category = category_label[i]
    location = extract_gps_info(image_path)
    if location:

        longitude = location[1]
        latitude = location[0]
        location_name, country = get_location_name(location[0], location[1])
        timestamp = location[2]
        longitudes.append(longitude)
        latitudes.append(latitude)
        timestamps.append(timestamp)
        location_names.append(location_name)
        countries.append(country)
        file_names.append(file)
        categories.append(category)
    else:
        print("\nNo geolocation information found in the photo." + file)




1/1 [==============================] - 0s 49ms/step


Possibly corrupted field Tag 0x001A in MakerNote IFD


1/1 [==============================] - 0s 49ms/step


Possibly corrupted field Tag 0x001A in MakerNote IFD


1/1 [==============================] - 0s 51ms/step


### 6. Create dataframe

In [47]:
image_dict = {'name': file_names, 'latitude':latitudes, 'longitude': longitudes, 'country':countries, 'location':location_names, 'time': timestamps, 'classification':categories}
image_df = pd.DataFrame(image_dict)

print(image_df)

             name   latitude  longitude                   country  \
0       berga.JPG  59.478528  18.315733                   Sverige   
1      cinisi.JPG  38.170647  13.107097                    Italia   
2       dubai.JPG  25.015797  55.725150  الإمارات العربية المتحدة   
3      gdansk.JPG  54.382308  18.636897                    Polska   
4    IMG-0562.JPG  60.127456  19.930411           Suomi / Finland   
..            ...        ...        ...                       ...   
111      nice.JPG  43.694325   7.261458                    France   
112     norge.JPG  59.033683   6.593531                     Norge   
113      prag.JPG  50.092025  14.406422                      None   
114     skane.JPG  55.337589  13.347958                   Sverige   
115     sthlm.JPG  59.345469  18.032781                   Sverige   

               location                 time classification  
0            Åkersberga  2020:11:07 17:20:51      buildings  
1                Cinisi  2022:06:19 19:41:52   

### 7. Load to csv-file

In [48]:
image_df.to_csv('image.csv', index=False)

Checking that latitude and longitude are correct.

In [49]:
image_paths = []
longitudes = []
latitudes = []
timestamps = []
location_names = []
countries = []
categories = []
file_names = []
category_label=['buildings','forest', 'glacier','mountain','sea','street']

file2 = "IMG_2064.jpg"

if True:
    image_path = (os.path.join(seg_our_path, file2))
    image_paths.append(image_path)
    img = cv2.imread(image_path)
    test1_proc = image2.load_img(image_path , target_size=(150,150))
    test1_proc_1 = image2.img_to_array(test1_proc)
    t1 = np.expand_dims(test1_proc ,axis=0)
    t1 = t1/255
    pred_classes = model.predict(t1)
    pred = np.argmax(pred_classes ,axis=1)
    for i in range(len(category_label)):
        if pred == i:
            category = category_label[i]
    location = extract_gps_info(image_path)
    if location:

        longitude = location[1]
        latitude = location[0]
        print(longitude, latitude)
        location_name, country = get_location_name(location[0], location[1])
        timestamp = location[2]
        longitudes.append(longitude)
        latitudes.append(latitude)
        timestamps.append(timestamp)
        location_names.append(location_name)
        countries.append(country)
        file_names.append(file)
        categories.append(category)
    else:
        print("\nNo geolocation information found in the photo." + file2)

1/1 [==============================] - 0s 46ms/step
-73.985625 40.748419444444444
